In [ ]:
!pip install -r requirements_dev.txt

In [ ]:
!wget "https://cloud.tsinghua.edu.cn/f/498512c3c1724558830d/?dl=1" -O parquet-train.arrow
!mkdir -p ./model
!wget "https://cloud.tsinghua.edu.cn/f/8bfd19e6cb1a4a289c1b/?dl=1" -O added_tokens.json
!wget "https://cloud.tsinghua.edu.cn/f/231ddebf6caf49b38ce8/?dl=1" -O config.json
!wget "https://cloud.tsinghua.edu.cn/f/79e402dcc503430db9a1/?dl=1" -O merges.txt
!wget "https://cloud.tsinghua.edu.cn/f/001e6641d7324635bc77/?dl=1" -O special_tokens_map.json
!wget "https://cloud.tsinghua.edu.cn/f/2d68e62358da4b7f94e6/?dl=1" -O tokenizer.json
!wget "https://cloud.tsinghua.edu.cn/f/5ebcd4f2380147e3bee8/?dl=1" -O tokenizer_config.json
!wget "https://cloud.tsinghua.edu.cn/f/34aa75355590497ba28b/?dl=1" -O vocab.json
!wget "https://cloud.tsinghua.edu.cn/f/cd59c04366674ab592b0/?dl=1" -O pytorch_model.bin

In [19]:
import os

In [62]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
!pip install git+https://github.com/zphang/transformers.git@neox20b
!pip install tensorboard==1.15.0
!pip install bitsandbytes-cuda117

In [ ]:
#!conda install cudatoolkit
!pip install bitsandbytes
!pip install --upgrade accelerate
!pip install --quiet git+https://github.com/huggingface/transformers.git

In [ ]:
!export PATH=/usr/local/cuda-11.7/bin${PATH:+:${PATH}}
!export LD_LIBRARY_PATH=/usr/local/cuda-11.7/lib64${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}

In [ ]:
!git clone https://github.com/timdettmers/bitsandbytes.git
!python bitsandbytes/setup.py install

In [31]:
timestamp = '2023-05-15 04:04:55'
model_name = 'dolly'
input_model = 'EleutherAI/pythia-2.8b'
checkpoint_dir_name = f"{model_name}_{timestamp}"
deepspeed_config = f'{os.getcwd()}/config/ds_z3_bf16_config.json'
local_output_dir = f"./{checkpoint_dir_name}"
dbfs_output_dir = ''
tensorboard_display_dir = f"{local_output_dir}/runs"
DATASET_FILE_PATH = f'{os.getcwd()}/parquet-train.arrow'
MODEL_PATH = f'{os.getcwd()}/model/'
CUDA_VISIBLE_DEVICES = 1
PATH = f"/usr/local/cuda/bin:{os.getenv('PATH')}"

In [73]:
!deepspeed --num_gpus=1 \
    --module training.trainer \
    --deepspeed "config/ds_z3_bf16_config.json" \
    --epochs 1 \
    --local-output-dir "2023-05-15 04:04:55" \
    --dbfs-output-dir "" \
    --per-device-train-batch-size 1 \
    --per-device-eval-batch-size 1 \
    --lr 1e-5 \
    --warmup-steps 50 \
    --input-model "EleutherAI/pythia-2.8b" \
    --logging-steps 1000 \
    --test-size 10

[2023-05-15 12:09:10,032] [WARNING] [runner.py:186:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2023-05-15 12:09:10,046] [INFO] [runner.py:550:main] cmd = /home/ec2-user/anaconda3/envs/pytorch_p39/bin/python3.9 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 --module --enable_each_rank_log=None training.trainer --deepspeed config/ds_z3_bf16_config.json --epochs 1 --local-output-dir 2023-05-15 04:04:55 --dbfs-output-dir  --per-device-train-batch-size 1 --per-device-eval-batch-size 1 --lr 1e-5 --warmup-steps 50 --input-model EleutherAI/pythia-2.8b --logging-steps 1000 --test-size 10
[2023-05-15 12:09:12,155] [INFO] [launch.py:142:main] WORLD INFO DICT: {'localhost': [0]}
[2023-05-15 12:09:12,155] [INFO] [launch.py:148:main] nnodes=1, num_local_procs=1, node_rank=0
[2023-05-15 12:09:12,155] [INFO] [launch.py:161:main] global_rank_mapping=defaultdict(<class 'list'>, {'localh

In [71]:
!nvidia-smi

Mon May 15 12:03:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   35C    P8    14W /  70W |      2MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------